## Company Sales Brochure Generator

- Create a product that can generate marketing brochures about a company
    - For prospective clients
    - For investors
    - For recruitment

- The Technology
    - Use Grok API/ OpenAI API
    - Use one-shot prompting
    - Stream back results and show with formatting
- We will be provided a company name and their primary website.




In [ ]:
# we will make the imports here
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown
from groq import Groq
import requests
from bs4 import BeautifulSoup

In [2]:
# fetch the API from the env file
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("API key not found")
    raise ValueError("API key not present in env fiel")
else:
    print("Found api key we are good to go")

Found api key we are good to go


In [8]:
# we will be given an URL and we need to fetch the content of that website also we need to scrape all the links present in the webpage
# then we need to create another function which would give us the relevant links

def getContentOfWebPage(url):
    response = requests.get(url)
    content_of_webpage = BeautifulSoup(response.text,'html.parser')
    return content_of_webpage.get_text()

getContentOfWebPage("https://edwarddonner.com")

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome - Edward Donner\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\nHomeConnect FourOutsmartAn arena that pits LLMs against each other in a battle of diplomacy and deviousnessAboutPosts\n\n\n\n\n\n\n\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (very amateur) and losing myself in Hacker News, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of Nebula.io. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt, acquired in 2021.\nWe work with groundbreaki

In [9]:
def getAllLinksFromAWebPage(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    # initialise an empty list
    list_of_tags = []
    for link in soup.find_all('a'):
        list_of_tags.append(link.get('href'))
    
    return list_of_tags

getAllLinksFromAWebPage("https://edwarddonner.com")

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

In [ ]:
# create an function that finds all the relevant tags
client = Groq(api_key=api_key)
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = getAllLinksFromAWebPage(url)
    user_prompt += "\n".join(links)
    return user_prompt

def extractAllRelevantTags(url):
    response = client.chat.completions.create(
        model = "llama-3.3-70b-versatile",
        messages= [
            {"role":"system", "content":f"{link_system_prompt}"},
            {"role":"user", "content":get_links_user_prompt(url)}
        ]
    )
    raw_content = (response.choices[0].message.content or "").strip()
    # Strip optional ```json fences before parsing
    if raw_content.startswith("```"):
        raw_content = raw_content[3:].strip()
        if raw_content.lower().startswith("json"):
            raw_content = raw_content[4:].strip()
        if raw_content.endswith("```"):
            raw_content = raw_content[:-3].strip()
    try:
        return json.loads(raw_content)
    except Exception:
        return {"links": []}

extractAllRelevantTags("https://edwarddonner.com")


'```json\n{\n    "links": [\n        {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},\n        {"type": "company website", "url": "https://edwarddonner.com/"},\n        {"type": "linkedin profile", "url": "https://www.linkedin.com/in/eddonner/"},\n        {"type": "twitter profile", "url": "https://twitter.com/edwarddonner"},\n        {"type": "facebook profile", "url": "https://www.facebook.com/edward.donner.52"}\n    ]\n}\n```'

In [13]:
## Brochure creation part

def fetchPageDataAndRelevantLinks(url):
    page_content = getContentOfWebPage(url)
    relevant_links = extractAllRelevantTags(url)
    result = f"## Landing Page:\n\n{page_content}\n## Relevant Links:\n"
    links = relevant_links.get("links", []) if isinstance(relevant_links, dict) else []
    for link in links:
        link_type = link.get("type", "link")
        link_url = link.get("url")
        if not link_url:
            continue
        result += f"\n\n### Link: {link_type}\n"
        result += getContentOfWebPage(link_url)
    return result

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetchPageDataAndRelevantLinks(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


def generateBrochure(company_name,url):
    response = client.chat.completions.create(
        model = "llama-3.3-70b-versatile",
        messages = [
            {"role":"user", "content":get_brochure_user_prompt(company_name,url)},
            {"role":"system", "content": f"{brochure_system_prompt}"}
        ]
    )
    content =  response.choices[0].message.content
    return Markdown(content)

generateBrochure(company_name="Huggingface", url = "https://huggingface.co/")

ecerPFN.cgiPFN.cgiPFN.cgi Jaune
## Introduction to Hugging Face
Hugging Face is a community-driven platform that enables the collaboration and development of artificial intelligence and machine learning models. The company provides a comprehensive ecosystem for data scientists, developers, and researchers to create, discover, and collaborate on machine learning projects.

## Our Mission
The mission of Hugging Face is to build the foundation of machine learning tooling with the community, fostering innovation, and facilitating the development of cutting-edge AI models.

## Products and Services
Hugging Face offers a range of products and services, including:
* **Models**: A vast repository of over 1 million models, including state-of-the-art AI models for various applications.
* **Datasets**: A collection of over 250,000 datasets, providing access to a wide range of data for machine learning tasks.
* **Spaces**: A platform for deploying and running machine learning applications, with over 400,000 applications available.
* **Compute**: A paid service offering optimized inference endpoints and GPU deployment options.
* **Enterprise**: A suite of solutions for teams and organizations, including single sign-on, priority support, and dedicated resources.

## Community and Culture
At Hugging Face, we value community and collaboration. Our platform is designed to facilitate the sharing of knowledge, models, and datasets, and to provide a space for developers, researchers, and data scientists to work together. We believe in open-source principles and strive to build a culture of transparency, inclusivity, and innovation.

## Customers and Partners
Hugging Face has a diverse range of customers and partners, including:
* **AI2**: A non-profit organization using Hugging Face for AI research and development.
* **Meta**: A technology company leveraging Hugging Face for AI-powered applications.
* **Amazon**: A multinational technology company using Hugging Face for machine learning and AI development.
* **Google**: A technology leader utilizing Hugging Face for AI research and innovation.
* **Intel**: A technology company partnering with Hugging Face for AI-powered solutions.

## Careers and Opportunities
Hugging Face offers a range of career opportunities for talented individuals in the fields of machine learning, software development, and data science. We are committed to building a diverse and inclusive team, and we encourage applicants from all backgrounds to join our community.

## Join Us
If you are passionate about artificial intelligence, machine learning, and community-driven development, we invite you to join us. Whether you are a developer, researcher, or simply interested in learning more about Hugging Face, we welcome you to explore our platform, join our community, and contribute to the future of AI.